# **Chapter 32: Database Testing Tools**

---

## **Introduction**

Database testing requires specialized tools that bridge the gap between application logic and data persistence layers. While manual verification through database clients is essential for exploratory testing, modern software development demands automated, repeatable, and scalable database testing solutions.

This chapter explores the comprehensive toolkit available for database testing—from traditional database management clients to modern containerized test databases and automated data generation frameworks. Understanding these tools is crucial for implementing **Shift-Left testing** practices, where data validation occurs early and continuously in the development pipeline.

---

## **32.1 Database Clients**

Database clients are software applications that allow testers to interact directly with database systems. They serve as the primary interface for manual verification, ad-hoc querying, and initial test data setup.

### **32.1.1 GUI-Based Database Clients**

**Universal Database Tools (Cross-Platform):**

**DBeaver (Community & Enterprise Editions)**
- **Industry Standard**: Open-source universal database tool supporting 80+ database systems
- **Testing Features**:
  - SQL editor with syntax highlighting and autocomplete
  - Query execution plan visualization (crucial for performance testing)
  - Data export/import for test data migration
  - ER diagram generation for schema validation
  - Mock data generation (limited but useful for quick tests)

```sql
-- Example: Using DBeaver for test data verification
-- Query to verify referential integrity after application operation
SELECT 
    o.order_id,
    o.customer_id,
    c.customer_name,
    COUNT(oi.item_id) as item_count,
    SUM(oi.quantity * oi.unit_price) as calculated_total,
    o.order_total as stored_total
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
LEFT JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.created_date >= '2026-01-01'
GROUP BY o.order_id, o.customer_id, c.customer_name, o.order_total
HAVING ABS(SUM(oi.quantity * oi.unit_price) - o.order_total) > 0.01;
-- This query validates that stored totals match calculated totals (data integrity test)
```

**DataGrip (JetBrains)**
- **Best for**: Developers and testers working in IntelliJ ecosystem
- **Features**: 
  - Smart code completion based on actual schema
  - Version control integration for SQL scripts
  - Compare database schemas (essential for migration testing)
  - Parameterized queries for test data variation

**Database-Specific Tools:**

| Database | Primary Tool | Key Testing Features |
|----------|--------------|---------------------|
| **MySQL/MariaDB** | MySQL Workbench | Visual explain plans, data modeling, migration tools |
| **PostgreSQL** | pgAdmin 4 | Query analyzer, server status monitoring, backup tools |
| **Microsoft SQL Server** | SSMS (SQL Server Management Studio) | Query execution plans, profiler for performance testing, data comparison |
| **Oracle** | SQL Developer | Unit testing framework for PL/SQL, data modeler |
| **MongoDB** | MongoDB Compass | Aggregation pipeline builder, schema analysis |

### **32.1.2 CLI (Command Line) Tools**

For automation and CI/CD integration, CLI tools are essential:

```bash
# PostgreSQL - psql for automated testing scripts
psql -h localhost -U testuser -d testdb -c "
  COPY (SELECT * FROM users WHERE created_at > NOW() - INTERVAL '1 day') 
  TO '/tmp/new_users.csv' WITH CSV HEADER;
"

# MySQL - mysql command line for test verification
mysql -u root -p -e "
  SELECT table_name, table_rows 
  FROM information_schema.tables 
  WHERE table_schema = 'production_db' 
  ORDER BY table_rows DESC;
"

# MongoDB - mongosh for document database testing
mongosh "mongodb://localhost:27017/testdb" --eval "
  db.orders.find({ status: 'pending', createdAt: { $lt: new Date(Date.now() - 7*24*60*60*1000) }}).count()
"
```

**Industry Best Practice**: Maintain a library of SQL scripts in version control (Git) for repeatable test data setup and verification queries. Use naming conventions:
- `setup_*.sql` - Test data insertion
- `verify_*.sql` - Validation queries
- `cleanup_*.sql` - Teardown scripts

### **32.1.3 Database Connection Management**

Secure connection handling is critical for testing across environments:

```python
# Python: Using environment variables for secure DB testing
import os
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

class DatabaseTestConfig:
    """
    Industry-standard configuration management for test databases
    Never hardcode credentials in test scripts
    """
    
    def __init__(self):
        self.host = os.getenv('TEST_DB_HOST', 'localhost')
        self.port = os.getenv('TEST_DB_PORT', '5432')
        self.database = os.getenv('TEST_DB_NAME', 'test_db')
        self.user = os.getenv('TEST_DB_USER', 'test_user')
        self.password = os.getenv('TEST_DB_PASSWORD', 'test_pass')
        self.ssl_mode = os.getenv('TEST_DB_SSL', 'prefer')
        
    def get_connection_string(self):
        """Generate PostgreSQL connection string with SSL"""
        return (f"postgresql://{self.user}:{self.password}@"
                f"{self.host}:{self.port}/{self.database}?"
                f"sslmode={self.ssl_mode}")
    
    def get_engine(self, pool_size=5):
        """
        Create SQLAlchemy engine with connection pooling
        Essential for performance testing scenarios
        """
        return create_engine(
            self.get_connection_string(),
            pool_size=pool_size,
            max_overflow=10,
            pool_timeout=30,
            pool_recycle=3600,
            echo=False  # Set to True for SQL query logging during debugging
        )

# Usage in test setup
config = DatabaseTestConfig()
engine = config.get_engine()
```

---

## **32.2 Automation with Database Connections**

Automated database testing requires programmatic interfaces that allow test scripts to execute SQL commands, verify results, and manage transactions.

### **32.2.1 Database Connectivity Standards**

**JDBC (Java Database Connectivity)** - Industry standard for Java-based testing:

```java
// Java: Robust database testing with JDBC and connection pooling
import java.sql.*;
import com.zaxxer.hikari.HikariConfig;
import com.zaxxer.hikari.HikariDataSource;

public class DatabaseTestAutomation {
    
    private HikariDataSource dataSource;
    
    public void setupConnectionPool() {
        // HikariCP - High-performance connection pooling for tests
        HikariConfig config = new HikariConfig();
        config.setJdbcUrl(System.getenv("TEST_DB_URL"));
        config.setUsername(System.getenv("TEST_DB_USER"));
        config.setPassword(System.getenv("TEST_DB_PASS"));
        config.setMaximumPoolSize(10);
        config.setMinimumIdle(2);
        config.setConnectionTimeout(30000);
        config.setLeakDetectionThreshold(60000); // Detect connection leaks
        
        // Test query validation
        config.setConnectionTestQuery("SELECT 1");
        
        this.dataSource = new HikariDataSource(config);
    }
    
    public boolean verifyTransactionIntegrity(String accountId, double expectedBalance) 
            throws SQLException {
        String sql = "SELECT balance, version FROM accounts WHERE account_id = ? " +
                     "FOR UPDATE"; // Lock row for concurrent testing
        
        try (Connection conn = dataSource.getConnection()) {
            conn.setAutoCommit(false); // Manual transaction control for testing
            
            try (PreparedStatement stmt = conn.prepareStatement(sql)) {
                stmt.setString(1, accountId);
                ResultSet rs = stmt.executeQuery();
                
                if (rs.next()) {
                    double actualBalance = rs.getDouble("balance");
                    int version = rs.getInt("version"); // Optimistic locking check
                    
                    // Verify balance with tolerance for floating point
                    boolean match = Math.abs(actualBalance - expectedBalance) < 0.001;
                    
                    if (!match) {
                        conn.rollback();
                        return false;
                    }
                    
                    conn.commit();
                    return true;
                }
            } catch (SQLException e) {
                conn.rollback();
                throw e;
            }
        }
        return false;
    }
    
    // Cleanup method
    public void teardown() {
        if (dataSource != null && !dataSource.isClosed()) {
            dataSource.close();
        }
    }
}
```

**Python DB-API 2.0** - Standard for Python database access:

```python
# Python: Comprehensive database testing framework
import psycopg2
from psycopg2.extras import RealDictCursor
from contextlib import contextmanager
import logging

class DatabaseTestFramework:
    """
    Production-ready database testing framework
    Implements best practices for connection handling and transaction management
    """
    
    def __init__(self, connection_params):
        self.params = connection_params
        self.logger = logging.getLogger(__name__)
        
    @contextmanager
    def get_connection(self, isolation_level=None):
        """
        Context manager for safe connection handling
        Ensures connections are always closed, even if exceptions occur
        """
        conn = None
        try:
            conn = psycopg2.connect(**self.params)
            
            if isolation_level:
                conn.set_session(isolation_level=isolation_level)
                
            yield conn
            
            # If we reach here without exception, commit
            if not conn.closed:
                conn.commit()
                
        except psycopg2.Error as e:
            self.logger.error(f"Database error: {e}")
            if conn and not conn.closed:
                conn.rollback()
            raise
        finally:
            if conn and not conn.closed:
                conn.close()
    
    def execute_test_query(self, query, params=None, fetch=True):
        """
        Execute test query with automatic resource management
        Returns results as list of dictionaries for easy assertion
        """
        with self.get_connection() as conn:
            with conn.cursor(cursor_factory=RealDictCursor) as cursor:
                self.logger.debug(f"Executing: {cursor.mogrify(query, params)}")
                cursor.execute(query, params)
                
                if fetch:
                    return cursor.fetchall()
                else:
                    return cursor.rowcount
    
    def verify_data_integrity(self, table_name, constraints):
        """
        Automated data integrity verification
        constraints: dict of column_name -> validation_function
        """
        violations = []
        
        with self.get_connection() as conn:
            with conn.cursor() as cursor:
                cursor.execute(f"SELECT * FROM {table_name}")
                rows = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]
                
                for row in rows:
                    row_dict = dict(zip(columns, row))
                    for column, validator in constraints.items():
                        if column in row_dict:
                            if not validator(row_dict[column]):
                                violations.append({
                                    'row_id': row_dict.get('id'),
                                    'column': column,
                                    'value': row_dict[column],
                                    'rule': validator.__name__
                                })
        
        return violations

# Usage example
def test_user_email_format():
    """Test that all emails follow proper format"""
    db = DatabaseTestFramework({
        'host': 'localhost',
        'database': 'test_db',
        'user': 'tester',
        'password': 'secure_pass'
    })
    
    # Define validation constraint
    def is_valid_email(email):
        import re
        return bool(re.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', email))
    
    violations = db.verify_data_integrity(
        'users',
        {'email': is_valid_email}
    )
    
    assert len(violations) == 0, f"Email validation failed: {violations}"
```

### **32.2.2 ORM (Object-Relational Mapping) Testing**

When testing applications using ORMs (Hibernate, SQLAlchemy, Entity Framework), test at both ORM and SQL levels:

```python
# SQLAlchemy testing example
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base
import pytest

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    email = Column(String(100))
    orders = relationship("Order", back_populates="user")

class Order(Base):
    __tablename__ = 'orders'
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    total_amount = Column(Integer)  # Stored in cents to avoid float issues
    user = relationship("User", back_populates="orders")

@pytest.fixture
def db_session():
    """Pytest fixture for database testing with rollback"""
    engine = create_engine('sqlite:///:memory:')  # In-memory for isolation
    Base.metadata.create_all(engine)
    
    Session = sessionmaker(bind=engine)
    session = Session()
    
    yield session
    
    session.rollback()  # Rollback all changes after test
    session.close()

def test_cascade_delete(db_session):
    """Verify that deleting user cascades to orders"""
    user = User(username='testuser', email='test@test.com')
    order = Order(total_amount=10000)
    user.orders.append(order)
    
    db_session.add(user)
    db_session.commit()
    
    # Verify initial state
    assert db_session.query(User).count() == 1
    assert db_session.query(Order).count() == 1
    
    # Delete user
    db_session.delete(user)
    db_session.commit()
    
    # Verify cascade
    assert db_session.query(User).count() == 0
    assert db_session.query(Order).count() == 0
```

---

## **32.3 Test Data Management (TDM)**

Test Data Management is a critical discipline ensuring that test environments have appropriate, secure, and representative data while complying with data privacy regulations (GDPR, CCPA, HIPAA).

### **32.3.1 Data Generation Strategies**

**Synthetic Data Generation:**

```python
# Using Faker library for realistic test data generation
from faker import Faker
from faker.providers import internet, credit_card, date_time
import random

class TestDataGenerator:
    """
    Generates realistic but synthetic test data
    Avoids using production data with PII (Personally Identifiable Information)
    """
    
    def __init__(self, locale='en_US', seed=None):
        self.fake = Faker(locale)
        self.fake.add_provider(internet)
        self.fake.add_provider(credit_card)
        self.fake.add_provider(date_time)
        
        if seed:
            Faker.seed(seed)  # Reproducible test data
        
    def generate_user_profile(self, user_type='standard'):
        """Generate complete user profile for testing"""
        profile = {
            'username': self.fake.user_name(),
            'email': self.fake.email(),
            'first_name': self.fake.first_name(),
            'last_name': self.fake.last_name(),
            'phone': self.fake.phone_number(),
            'address': {
                'street': self.fake.street_address(),
                'city': self.fake.city(),
                'state': self.fake.state(),
                'zip': self.fake.zipcode(),
                'country': self.fake.country()
            },
            'date_of_birth': self.fake.date_of_birth(minimum_age=18, maximum_age=90),
            'registration_date': self.fake.date_time_this_decade(),
            'is_active': random.choice([True, False]),
            'user_type': user_type,
            'credit_limit': random.randint(1000, 50000) if user_type == 'premium' else 0
        }
        return profile
    
    def generate_transaction(self, user_id=None):
        """Generate financial transaction for testing calculations"""
        return {
            'transaction_id': self.fake.uuid4(),
            'user_id': user_id or random.randint(1, 10000),
            'amount': round(random.uniform(1.00, 10000.00), 2),
            'currency': random.choice(['USD', 'EUR', 'GBP', 'JPY']),
            'timestamp': self.fake.date_time_this_year(),
            'status': random.choice(['completed', 'pending', 'failed', 'refunded']),
            'payment_method': random.choice(['credit_card', 'debit_card', 'paypal', 'crypto']),
            'merchant_id': self.fake.company()
        }
    
    def bulk_insert_data(self, db_connection, table_name, count=1000):
        """
        Efficient bulk data insertion for performance testing
        Uses execute_batch for PostgreSQL or executemany for others
        """
        import psycopg2.extras
        
        data = [self.generate_user_profile() for _ in range(count)]
        
        query = """
            INSERT INTO users (username, email, first_name, last_name, created_at)
            VALUES (%(username)s, %(email)s, %(first_name)s, %(last_name)s, %(registration_date)s)
        """
        
        with db_connection.cursor() as cursor:
            psycopg2.extras.execute_batch(cursor, query, data)
            db_connection.commit()
```

### **32.3.2 Data Masking and Subsetting**

When using production-like data, sensitive information must be masked:

```python
# Data masking for compliance testing
import hashlib
from cryptography.fernet import Fernet

class DataMasking:
    """
    GDPR/HIPAA compliant data masking for test environments
    """
    
    def __init__(self, encryption_key=None):
        self.key = encryption_key or Fernet.generate_key()
        self.cipher = Fernet(self.key)
    
    def mask_email(self, email):
        """Preserve domain, mask local part"""
        if '@' not in email:
            return self.hash_value(email)
        local, domain = email.split('@')
        masked_local = local[:2] + '***' + local[-1] if len(local) > 3 else '***'
        return f"{masked_local}@{domain}"
    
    def mask_credit_card(self, card_number):
        """PCI DSS compliant masking - show only last 4 digits"""
        clean = str(card_number).replace(' ', '').replace('-', '')
        if len(clean) >= 4:
            return '*' * (len(clean) - 4) + clean[-4:]
        return '****'
    
    def tokenize(self, value):
        """Reversible tokenization for values that need to be recovered"""
        return self.cipher.encrypt(value.encode()).decode()
    
    def detokenize(self, token):
        """Recover original value"""
        return self.cipher.decrypt(token.encode()).decode()
    
    def hash_value(self, value, salt=None):
        """One-way hashing for passwords/SSNs"""
        salted = f"{value}{salt}" if salt else value
        return hashlib.sha256(salted.encode()).hexdigest()

# Database subsetting - extract representative sample
def create_test_subset(source_db, target_db, percentage=10):
    """
    Create a subset of production data for testing
    Maintains referential integrity while reducing volume
    """
    subset_strategy = {
        'users': f"ORDER BY RANDOM() LIMIT (SELECT COUNT(*) * {percentage/100} FROM users)",
        'orders': "WHERE user_id IN (SELECT id FROM users)",  # Referential integrity
        'order_items': "WHERE order_id IN (SELECT id FROM orders)",
        'products': "WHERE id IN (SELECT product_id FROM order_items)"  # Only referenced products
    }
    
    for table, condition in subset_strategy.items():
        copy_query = f"""
            CREATE TABLE {target_db}.{table} AS 
            SELECT * FROM {source_db}.{table} 
            {condition};
        """
        # Execute copy...
```

### **32.3.3 Enterprise TDM Tools**

For large organizations, commercial TDM solutions provide:

| Tool | Key Features | Best For |
|------|--------------|----------|
| **Delphix** | Virtual data copies, masking, self-service | Large enterprises, complex data landscapes |
| **Informatica TDM** | Data generation, subsetting, masking | Regulatory compliance, GDPR |
| **CA TDM (Broadcom)** | Synthetic data, test matching | Agile teams, CI/CD integration |
| **IBM InfoSphere Optim** | Archive, subset, mask | Legacy systems, mainframe |
| **K2View** | Micro-databases, privacy | Real-time test data provisioning |

**Open Source Alternative**: **PostgreSQL Anonymizer** or **MySQL Data Masking** plugins for basic masking needs.

---

## **32.4 Database Mocking**

Database mocking isolates tests from external dependencies, ensuring tests are fast, reliable, and don't require running database servers.

### **32.4.1 In-Memory Databases**

```java
// Java: Using H2 in-memory database for unit testing
import org.junit.jupiter.api.*;
import java.sql.*;

public class InMemoryDatabaseTest {
    
    private static Connection conn;
    
    @BeforeAll
    public static void setup() throws SQLException {
        // H2 in-memory database - fast, isolated, no external dependencies
        String url = "jdbc:h2:mem:testdb;DB_CLOSE_DELAY=-1;MODE=PostgreSQL";
        conn = DriverManager.getConnection(url, "sa", "");
        
        // Create schema
        Statement stmt = conn.createStatement();
        stmt.execute("""
            CREATE TABLE IF NOT EXISTS inventory (
                product_id VARCHAR(50) PRIMARY KEY,
                quantity INT NOT NULL CHECK (quantity >= 0),
                last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """);
    }
    
    @Test
    public void testInventoryUpdate() throws SQLException {
        // Insert test data
        PreparedStatement insert = conn.prepareStatement(
            "INSERT INTO inventory VALUES (?, ?, ?)"
        );
        insert.setString(1, "PROD-001");
        insert.setInt(2, 100);
        insert.setTimestamp(3, new Timestamp(System.currentTimeMillis()));
        insert.executeUpdate();
        
        // Test business logic: decrement inventory
        PreparedStatement update = conn.prepareStatement(
            "UPDATE inventory SET quantity = quantity - ? WHERE product_id = ?"
        );
        update.setInt(1, 5);
        update.setString(2, "PROD-001");
        int affected = update.executeUpdate();
        
        Assertions.assertEquals(1, affected);
        
        // Verify state
        Statement stmt = conn.createStatement();
        ResultSet rs = stmt.executeQuery(
            "SELECT quantity FROM inventory WHERE product_id = 'PROD-001'"
        );
        Assertions.assertTrue(rs.next());
        Assertions.assertEquals(95, rs.getInt("quantity"));
    }
    
    @AfterAll
    public static void teardown() throws SQLException {
        if (conn != null) conn.close();
        // H2 in-memory DB destroyed automatically
    }
}
```

### **32.4.2 Testcontainers for Integration Testing**

**Testcontainers** is the industry standard for database integration testing, providing real databases in Docker containers:

```python
# Python: Testcontainers for real database testing
import pytest
from testcontainers.postgres import PostgresContainer
from testcontainers.mysql import MySqlContainer
import psycopg2

@pytest.fixture(scope="module")
def postgres_db():
    """
    Spins up real PostgreSQL in Docker container
    Provides full PostgreSQL features unlike SQLite
    """
    with PostgresContainer("postgres:15-alpine") as postgres:
        # Wait for database to be ready
        conn = psycopg2.connect(
            host=postgres.get_container_host_ip(),
            port=postgres.get_exposed_port(5432),
            user=postgres.username,
            password=postgres.password,
            dbname=postgres.dbname
        )
        
        # Setup schema
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE events (
                id SERIAL PRIMARY KEY,
                event_type VARCHAR(50) NOT NULL,
                payload JSONB,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        conn.commit()
        
        yield conn
        
        # Cleanup automatic via context manager

def test_json_operations(postgres_db):
    """Test PostgreSQL-specific JSON operations"""
    cursor = postgres_db.cursor()
    
    # Insert JSON data
    cursor.execute("""
        INSERT INTO events (event_type, payload) 
        VALUES ('user_action', '{"user_id": 123, "action": "login"}')
    """)
    
    # Query JSON
    cursor.execute("""
        SELECT payload->>'user_id' as user_id 
        FROM events 
        WHERE event_type = 'user_action'
    """)
    
    result = cursor.fetchone()
    assert result[0] == '123'
```

### **32.4.3 Mocking Database Connections**

For unit testing without any database:

```python
# Python: Mocking database with unittest.mock
from unittest.mock import Mock, MagicMock, patch
import pytest

def test_with_mocked_db():
    """
    Unit test with completely mocked database
    Fastest execution, no I/O
    """
    # Mock connection and cursor
    mock_conn = Mock()
    mock_cursor = Mock()
    
    # Configure mock behavior
    mock_cursor.fetchall.return_value = [
        {'id': 1, 'username': 'alice', 'status': 'active'},
        {'id': 2, 'username': 'bob', 'status': 'inactive'}
    ]
    mock_cursor.rowcount = 2
    
    mock_conn.cursor.return_value.__enter__ = Mock(return_value=mock_cursor)
    mock_conn.cursor.return_value.__exit__ = Mock(return_value=False)
    
    # Inject mock into application code
    from myapp import UserService
    service = UserService(mock_conn)
    
    users = service.get_active_users()
    
    # Verify interactions
    mock_cursor.execute.assert_called_once()
    assert len(users) == 2
    assert all(u['status'] == 'active' for u in users)

# Java: Mockito for database testing
/*
@ExtendWith(MockitoExtension.class)
class UserRepositoryTest {
    
    @Mock
    private Connection mockConnection;
    
    @Mock
    private PreparedStatement mockStatement;
    
    @Mock
    private ResultSet mockResultSet;
    
    @Test
    void testFindUserById() throws SQLException {
        // Arrange
        when(mockConnection.prepareStatement(anyString()))
            .thenReturn(mockStatement);
        when(mockStatement.executeQuery()).thenReturn(mockResultSet);
        when(mockResultSet.next()).thenReturn(true, false);
        when(mockResultSet.getString("username")).thenReturn("testuser");
        
        UserRepository repo = new UserRepository(mockConnection);
        
        // Act
        User user = repo.findById(1L);
        
        // Assert
        assertEquals("testuser", user.getUsername());
        verify(mockStatement).setLong(1, 1L);
    }
}
*/
```

---

## **32.5 Code Examples for Database Testing**

### **32.5.1 Complete Test Suite Example**

```python
# Comprehensive database testing suite
import pytest
import psycopg2
from psycopg2.extras import RealDictCursor
from contextlib import contextmanager

class DatabaseTestSuite:
    """
    Production-ready database testing patterns
    """
    
    def __init__(self, db_config):
        self.config = db_config
        self.setup_scripts = []
        self.teardown_scripts = []
    
    @contextmanager
    def transaction_scope(self):
        """Context manager for test transactions with automatic rollback"""
        conn = psycopg2.connect(**self.config)
        conn.set_session(autocommit=False)
        try:
            yield conn
        finally:
            conn.rollback()  # Always rollback after test
            conn.close()
    
    def setup_test_data(self, conn, fixture_name):
        """Load SQL fixtures"""
        with open(f'fixtures/{fixture_name}.sql', 'r') as f:
            sql = f.read()
            with conn.cursor() as cursor:
                cursor.execute(sql)
                conn.commit()
    
    def assert_row_count(self, conn, table_name, expected_count, where_clause=None):
        """Assertion helper for row counts"""
        query = f"SELECT COUNT(*) FROM {table_name}"
        if where_clause:
            query += f" WHERE {where_clause}"
        
        with conn.cursor() as cursor:
            cursor.execute(query)
            actual = cursor.fetchone()[0]
            assert actual == expected_count, \
                f"Expected {expected_count} rows in {table_name}, found {actual}"
    
    def assert_foreign_key_integrity(self, conn, parent_table, child_table, fk_column):
        """Verify referential integrity"""
        query = f"""
            SELECT COUNT(*) FROM {child_table} c
            LEFT JOIN {parent_table} p ON c.{fk_column} = p.id
            WHERE p.id IS NULL AND c.{fk_column} IS NOT NULL
        """
        with conn.cursor() as cursor:
            cursor.execute(query)
            orphan_count = cursor.fetchone()[0]
            assert orphan_count == 0, \
                f"Found {orphan_count} orphaned records in {child_table}"

# Pytest fixtures for reuse
@pytest.fixture
def db_suite():
    config = {
        'host': 'localhost',
        'dbname': 'test_db',
        'user': 'test_user',
        'password': 'test_pass'
    }
    return DatabaseTestSuite(config)

def test_order_workflow(db_suite):
    """End-to-end test of order creation workflow"""
    with db_suite.transaction_scope() as conn:
        # Setup
        db_suite.setup_test_data(conn, 'initial_users')
        
        # Execute business operation
        with conn.cursor() as cursor:
            # Create order
            cursor.execute("""
                INSERT INTO orders (user_id, total_amount, status)
                VALUES ((SELECT id FROM users LIMIT 1), 100.00, 'pending')
                RETURNING id
            """)
            order_id = cursor.fetchone()[0]
            
            # Add items
            cursor.execute("""
                INSERT INTO order_items (order_id, product_id, quantity, price)
                VALUES (%s, 1, 2, 50.00)
            """, (order_id,))
            
            # Update inventory
            cursor.execute("""
                UPDATE inventory 
                SET quantity = quantity - 2 
                WHERE product_id = 1
            """)
        
        # Verify
        db_suite.assert_row_count(conn, 'orders', 1, f"id = {order_id}")
        db_suite.assert_row_count(conn, 'order_items', 1, f"order_id = {order_id}")
        
        # Verify inventory decrement
        with conn.cursor() as cursor:
            cursor.execute("SELECT quantity FROM inventory WHERE product_id = 1")
            qty = cursor.fetchone()[0]
            assert qty == 98  # Assuming started with 100

def test_concurrent_updates(db_suite):
    """Test handling of concurrent database updates"""
    import threading
    import queue
    
    results = queue.Queue()
    
    def update_balance(account_id, amount):
        try:
            with db_suite.transaction_scope() as conn:
                with conn.cursor() as cursor:
                    # Optimistic locking pattern
                    cursor.execute("""
                        UPDATE accounts 
                        SET balance = balance + %s, 
                            version = version + 1
                        WHERE id = %s 
                        AND version = (SELECT version FROM accounts WHERE id = %s)
                        RETURNING version
                    """, (amount, account_id, account_id))
                    
                    if cursor.rowcount == 0:
                        results.put(('conflict', account_id))
                    else:
                        results.put(('success', account_id))
        except Exception as e:
            results.put(('error', str(e)))
    
    # Simulate concurrent updates
    threads = []
    for i in range(5):
        t = threading.Thread(target=update_balance, args=(1, 100))
        threads.append(t)
        t.start()
    
    for t in threads:
        t.join()
    
    # Analyze results - should have some conflicts
    outcomes = list(results.queue)
    successes = [o for o in outcomes if o[0] == 'success']
    conflicts = [o for o in outcomes if o[0] == 'conflict']
    
    assert len(successes) > 0, "At least one update should succeed"
    print(f"Successful updates: {len(successes)}, Conflicts: {len(conflicts)}")
```

### **32.5.2 Stored Procedure Testing**

```sql
-- Testing stored procedures (PostgreSQL example)
-- File: test_stored_procedures.sql

-- Test Framework Setup
CREATE OR REPLACE FUNCTION run_test(test_name TEXT, expected_result BOOLEAN, actual_result BOOLEAN)
RETURNS VOID AS $$
BEGIN
    IF expected_result = actual_result THEN
        RAISE NOTICE 'PASS: %', test_name;
    ELSE
        RAISE EXCEPTION 'FAIL: % - Expected %, Got %', test_name, expected_result, actual_result;
    END END IF;
END;
$$ LANGUAGE plpgsql;

-- Test: Calculate Order Total
CREATE OR REPLACE FUNCTION test_calculate_order_total()
RETURNS VOID AS $$
DECLARE
    v_order_id INT;
    v_result DECIMAL;
BEGIN
    -- Setup
    INSERT INTO orders (customer_id, order_date) VALUES (1, NOW()) RETURNING id INTO v_order_id;
    INSERT INTO order_items (order_id, product_id, quantity, unit_price) 
    VALUES (v_order_id, 1, 2, 25.00), (v_order_id, 2, 1, 50.00);
    
    -- Execute
    SELECT calculate_order_total(v_order_id) INTO v_result;
    
    -- Assert
    PERFORM run_test('Order Total Calculation', v_result = 100.00, ABS(v_result - 100.00) < 0.01);
    
    -- Teardown
    ROLLBACK;  -- Automatic cleanup
END;
$$ LANGUAGE plpgsql;

-- Execute tests
SELECT test_calculate_order_total();
```

### **32.5.3 Performance Baseline Testing**

```python
# Database performance testing
import time
import statistics
from contextlib import contextmanager

class DatabasePerformanceTest:
    """
    Performance testing utilities for databases
    """
    
    def __init__(self, db_connection):
        self.conn = db_connection
        self.measurements = []
    
    @contextmanager
    def measure_query_time(self, query_name):
        """Context manager to measure query execution time"""
        start = time.perf_counter()
        yield
        end = time.perf_counter()
        duration = (end - start) * 1000  # Convert to milliseconds
        self.measurements.append({
            'query': query_name,
            'duration_ms': duration,
            'timestamp': time.time()
        })
    
    def run_load_test(self, query_func, iterations=100, concurrency=1):
        """
        Execute query multiple times to establish baseline
        """
        from concurrent.futures import ThreadPoolExecutor
        
        def single_execution(i):
            with self.measure_query_time(f"iteration_{i}"):
                return query_func()
        
        with ThreadPoolExecutor(max_workers=concurrency) as executor:
            results = list(executor.map(single_execution, range(iterations)))
        
        return self._analyze_results()
    
    def _analyze_results(self):
        """Calculate performance metrics"""
        times = [m['duration_ms'] for m in self.measurements]
        
        return {
            'count': len(times),
            'mean_ms': statistics.mean(times),
            'median_ms': statistics.median(times),
            'stdev_ms': statistics.stdev(times) if len(times) > 1 else 0,
            'min_ms': min(times),
            'max_ms': max(times),
            'p95_ms': sorted(times)[int(len(times)*0.95)] if times else 0,
            'p99_ms': sorted(times)[int(len(times)*0.99)] if times else 0
        }
    
    def assert_performance(self, metric_name, max_acceptable_ms):
        """Assert that performance meets SLA"""
        analysis = self._analyze_results()
        actual = analysis.get(metric_name, 0)
        
        assert actual <= max_acceptable_ms, \
            f"Performance regression: {metric_name}={actual}ms exceeds threshold {max_acceptable_ms}ms"

# Usage
def test_user_search_performance(db_connection):
    perf_test = DatabasePerformanceTest(db_connection)
    
    def search_query():
        cursor = db_connection.cursor()
        cursor.execute("""
            SELECT u.*, COUNT(o.id) as order_count 
            FROM users u 
            LEFT JOIN orders o ON u.id = o.user_id 
            WHERE u.email LIKE '%@example.com%' 
            GROUP BY u.id 
            LIMIT 100
        """)
        return cursor.fetchall()
    
    # Run 100 iterations, 5 concurrent threads
    results = perf_test.run_load_test(search_query, iterations=100, concurrency=5)
    
    # Assert against SLA (e.g., 95th percentile must be under 100ms)
    perf_test.assert_performance('p95_ms', 100)
    
    print(f"Query performance: Mean={results['mean_ms']:.2f}ms, P95={results['p95_ms']:.2f}ms")
```

---

## **Chapter Summary**

### **Key Takeaways:**

**Database Clients (32.1):**
- **DBeaver** and **DataGrip** are industry-standard GUI tools for manual database verification and exploratory testing
- CLI tools enable automation and CI/CD integration
- Always use environment variables for credentials; never hardcode in test scripts

**Automation Connectivity (32.2):**
- **Connection Pooling** (HikariCP for Java, SQLAlchemy for Python) is essential for performance testing
- Use **context managers** to ensure connections are properly closed
- Implement **transaction rollback** after tests to maintain clean state

**Test Data Management (32.3):**
- **Synthetic data generation** (Faker) is preferred over production data for privacy compliance
- **Data masking** is mandatory when using production-derived data (GDPR/HIPAA)
- **Subsetting** reduces test environment size while maintaining referential integrity
- Enterprise tools (Delphix, Informatica) provide advanced TDM capabilities for large organizations

**Database Mocking (32.4):**
- **H2/SQLite** in-memory databases provide fast unit testing for simple scenarios
- **Testcontainers** offer the best balance—isolated real databases without infrastructure overhead
- **Mocking** (Mockito/Mock) is fastest but only suitable for unit testing logic, not SQL validation

**Best Practices:**
1. **Isolation**: Each test should use independent transactions or database instances
2. **Reproducibility**: Use seed values for random data generation to enable debugging
3. **Cleanup**: Implement teardown methods; use `TRUNCATE` over `DELETE` for speed
4. **Version Control**: Store schema migrations and test data SQL in Git
5. **Performance**: Establish baselines and fail tests on regression (SLA violations)

### **Tools Reference:**
- **GUI**: DBeaver (universal), pgAdmin (PostgreSQL), MySQL Workbench, SSMS (SQL Server)
- **Python**: `psycopg2`, `pymysql`, `sqlalchemy`, `faker`, `testcontainers`
- **Java**: JDBC, H2, Testcontainers, HikariCP
- **TDM**: Delphix, Informatica TDM, PostgreSQL Anonymizer

---

## **📖 Next Chapter: Chapter 33 - Performance Testing Fundamentals**

Now that you understand how to test database functionality and manage test data, **Chapter 33** will introduce you to **Performance Testing**—ensuring your applications meet speed, scalability, and stability requirements under load.

In **Chapter 33**, you will master:

- **Performance Testing Types**: Load Testing, Stress Testing, Spike Testing, Endurance Testing, and Scalability Testing—understanding when to apply each
- **Performance Metrics**: Response time, throughput, latency, error rates, and resource utilization (CPU, memory, disk I/O)
- **Performance Testing Strategy**: Defining SLAs (Service Level Agreements), establishing baselines, and identifying performance bottlenecks
- **The Performance Testing Process**: From planning and script creation to execution and analysis
- **Key Concepts**: Think time, pacing, ramp-up/ramp-down strategies, and correlation
- **Industry Standards**: Web performance metrics (Apdex scores), percentiles (P95, P99), and golden signals (latency, traffic, errors, saturation)

This chapter will transition you from functional verification to **non-functional testing**, teaching you how to validate that your database queries, APIs, and applications perform efficiently under real-world load conditions.

**Continue to Chapter 33 to learn how to ensure your applications scale and perform under pressure!**